In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/碩一上課堂/1131_dataMining/dataMining/HW5/movieRating.csv')

In [10]:
data.head()

,TrainDataID,UserID,MovieID,Rating
335378,335379,2794,1643,4
743547,743548,4761,25,2
416126,416127,844,3673,1
570976,570977,3974,1683,3
635909,635910,2927,3478,3


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Add
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error

In [5]:
# 打亂資料集
data = shuffle(data, random_state=42)

# 分割訓練集與測試集 (80% 訓練集, 20% 測試集)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 獲取唯一使用者和電影數量
n_users = data['UserID'].nunique()
n_movies = data['MovieID'].nunique()

# 準備訓練和測試資料
train_user = train_data['UserID'].values
train_movie = train_data['MovieID'].values
train_rating = train_data['Rating'].values

test_user = test_data['UserID'].values
test_movie = test_data['MovieID'].values
test_rating = test_data['Rating'].values

In [6]:
# 建立矩陣分解模型
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

user_embedding = Embedding(input_dim=n_users + 1, output_dim=50)(user_input)
movie_embedding = Embedding(input_dim=n_movies + 1, output_dim=50)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vec, movie_vec])

user_bias = Embedding(input_dim=n_users + 1, output_dim=1)(user_input)
movie_bias = Embedding(input_dim=n_movies + 1, output_dim=1)(movie_input)

user_bias_vec = Flatten()(user_bias)
movie_bias_vec = Flatten()(movie_bias)

output = Add()([dot_product, user_bias_vec, movie_bias_vec])

# 編譯模型
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# 訓練模型
model.fit(
    [train_user, train_movie],
    train_rating,
    validation_data=([test_user, test_movie], test_rating),
    epochs=5,
    batch_size=1024,
    verbose=1
)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 11.0304 - val_loss: 1.3557
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.1435 - val_loss: 0.9245
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8915 - val_loss: 0.8676
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8507 - val_loss: 0.8464
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8154 - val_loss: 0.8317


In [7]:
# 預測並計算 MAE
test_predictions = model.predict([test_user, test_movie])
mae = mean_absolute_error(test_rating, test_predictions)

print(f"Mean Absolute Error (MAE): {mae}")

5625/5625 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
Mean Absolute Error (MAE): 0.7190045403204066


In [8]:
# 打亂資料集
data = shuffle(data, random_state=42)

# 分割訓練集與測試集 (80% 訓練集, 20% 測試集)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 獲取唯一使用者和電影數量
n_users = data['UserID'].nunique()
n_movies = data['MovieID'].nunique()

# 準備訓練和測試資料
train_user = train_data['UserID'].values
train_movie = train_data['MovieID'].values
train_rating = train_data['Rating'].values

test_user = test_data['UserID'].values
test_movie = test_data['MovieID'].values
test_rating = test_data['Rating'].values

# 建立矩陣分解模型
user_input = Input(shape=(1,))
movie_input = Input(shape=(1,))

# 增加 embedding 維度到 100
user_embedding = Embedding(input_dim=n_users + 1, output_dim=100)(user_input)
movie_embedding = Embedding(input_dim=n_movies + 1, output_dim=100)(movie_input)

user_vec = Flatten()(user_embedding)
movie_vec = Flatten()(movie_embedding)

dot_product = Dot(axes=1)([user_vec, movie_vec])

user_bias = Embedding(input_dim=n_users + 1, output_dim=1)(user_input)
movie_bias = Embedding(input_dim=n_movies + 1, output_dim=1)(movie_input)

user_bias_vec = Flatten()(user_bias)
movie_bias_vec = Flatten()(movie_bias)

output = Add()([dot_product, user_bias_vec, movie_bias_vec])

# 編譯模型
model = Model(inputs=[user_input, movie_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# 增加訓練輪數到 10
model.fit(
    [train_user, train_movie],
    train_rating,
    validation_data=([test_user, test_movie], test_rating),
    epochs=10,
    batch_size=1024,
    verbose=1
)

# 預測並計算 MAE
test_predictions = model.predict([test_user, test_movie])
mae = mean_absolute_error(test_rating, test_predictions)

print(f"Mean Absolute Error (MAE): {mae}")

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 10.7721 - val_loss: 1.1109
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9929 - val_loss: 0.8806
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8567 - val_loss: 0.8461
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8061 - val_loss: 0.8138
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7574 - val_loss: 0.7990
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7202 - val_loss: 0.8086
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6898 - val_loss: 0.7929
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.6461 - val_loss: 0.7898
Epoch 9/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6033 - val_loss: 0.7995
Epoch 10/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5639 - val_loss: 0.7975
5625/5625 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
Mean Absolute Error (MAE): 0.697056373377975


In [ ]:
|